# Modelo de Segregación de Schelling (Discreto)

- Dos diferentes tipos de agentes distribuidos en un espacio finito de dimensión 2.
- En cada iteración se elige un agente al azar se ve a sus vecinos y si cierta fracción de ellos son del otro tipo, salta a otro lugar aleatoriamente escogido en el espacio.


## Agente del modelo de Schelling

In [1]:
class AgenteSchelling:
    """
    Agente del modelo de Schelling
    Atributos:
        tipo/color: "rojo" "azul"
        
    """
    def __init__(self,color,pos, tolerancia):
        self.color = color
        self.pos = pos
        self.tolerancia = tolerancia

### Creación de un agente

In [2]:
agente1 = AgenteSchelling('red',(0,1),0.6)

### Acceder a un atributo del agente

In [3]:
agente1.color

'red'

In [4]:
agente1.pos

(0, 1)

In [5]:
agente1.tolerancia

0.6

### Importamos la clase Agent y Model del modulo Mesa

In [6]:
from mesa.space import SingleGrid #MultiGrid, Space

In [7]:
from mesa import Agent, Model

In [16]:
class AgenteSchelling(Agent):
    """
    Agente del modelo de Schelling
    Esta clase implementa un agente para el modelo de segregación de Schelling
    Atributos:
        tipo/color: "rojo" "azul"
        pos: posición
        tolerancia: porcentaje de vecinos al menos iguales al agente
        modelo: modelo del 
        
    """
    def __init__(self,color,pos, tolerancia, model):
        # Aqui van los atributos del agente
        self.color = color
        self.pos = pos
        self.tolerancia = tolerancia
        self.model  = model
        
    def step(self):
        
        #Buscar ccuales son los vecinos
        vecinos = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        return vecinos
#         #Contar cuantos vecinos son iguales
#         self.
        
        
class ModeloSchelling(Model):
    """
    Modelo de Schelling
    ancho: ancho de la malla
    """
    def __init__(self,ancho,largo):
        self.ancho = ancho
        self.largo = largo
        self.grid = SingleGrid(ancho, largo, torus=True)

### Creamos nuestro modelo

In [17]:
modelo_prueba = ModeloSchelling(10,10)

# Atributos del modelo
print(modelo_juguete.ancho)
print(modelo_juguete.largo)

10
10


In [18]:
agente1 = AgenteSchelling('rojo',(0,0), 0.4,modelo_prueba)

# Atributos del agente
print(agente1.color)
print(agente1.pos)
print(agente1.tolerancia)

rojo
(0, 0)
0.4


### Muestrar la malla

In [22]:
modelo_prueba.grid.grid

[[None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None]]